# Setup
```bash
jupyter nbextension enable --py --sys-prefix ipyvuetify
```

In [2]:
import requests
import json
import pathlib

import ipyvuetify as v
from ipywidgets import jslink

PTV_API = pathlib.Path("key.txt").read_text()


In [91]:
def get_coordinates(place_query):
    response = requests.get(
        f"https://api.myptv.com/geocoding/v1/locations/by-text?searchText={place_query}&apiKey={PTV_API}"
    )
    location = response.json()["locations"][0]
    return {
        "longitude": location["referencePosition"]["longitude"],
        "latitude": location["referencePosition"]["latitude"],
        "country": location["address"]["countryName"],
        "city": location["address"]["city"],
    }


In [95]:
def get_photovoltaic_data(latitude, longitude):
    response = requests.get(
        f"https://re.jrc.ec.europa.eu/api/MRcalc?lat={latitude}&lon={longitude}&horirrad=1&outputformat=json"
    )

    return response.json()


In [101]:
country = v.TextField(label="Land", v_model="", disabled=True)
city = v.TextField(label="Stadt", v_model="", disabled=True)
latitude = v.TextField(label="Latitude", v_model="", disabled=True)
longitude = v.TextField(label="Longitude", v_model="", disabled=True)
coordinate_row = v.Row(
    children=[
        country,
        city,
        latitude,
        longitude,
    ]
)
adress = v.TextField(label="Adresse", v_model="", prepend_icon="mdi-map-marker")


def get_adress(widget, event, data):
    coordinates = get_coordinates(adress.v_model)
    country.v_model = coordinates["country"]
    city.v_model = coordinates["city"]
    latitude.v_model = coordinates["latitude"]
    longitude.v_model = coordinates["longitude"]
    data_text.v_model = (
        f'{get_photovoltaic_data(coordinates["latitude"],coordinates["longitude"])}'
    )


search_adress_btn = v.Btn(children=["such"])
search_adress_btn.on_event("click", get_adress)

data_text = v.Textarea(v_model="", disabled=True)
v.Container(
    children=[coordinate_row, v.Row(children=[adress, search_adress_btn]), data_text]
)


Container(children=[Row(children=[TextField(disabled=True, label='Land', v_model=''), TextField(disabled=True,…